This notebook helps me have a good overview of my code and its directly corelated output.
I will try gathering as much information as possible from "apartment guides" and save the content in a csv file for further data analysis

In [ ]:
import requests 
from bs4 import BeautifulSoup 


def addUrls(url, listUrl): 
    #open with GET method 
    resp=requests.get(url)         
    if resp.status_code==200: 
        print("Successfully opened the web page") 
        listUrl.append(url)
    else:
        print("Did not work")
    soup=BeautifulSoup(resp.text,'html.parser')
    l=soup.find("a",{"class":"_2glka"})   #_2glka has to be adjusted based on website; here this is Next Page
    if (l.get('href') == ""):
        print('Done adding')
    else:                
        addUrls(l.get('href'), listUrl) 




In [ ]:
def addSubSites(url, subSiteList): 
    #open with GET method 
    resp=requests.get(url)         
    if resp.status_code==200: 
        print("Successfully opened the web page") 
        #listUrl.append(url)
    else:
        print("Did not work")
    soup=BeautifulSoup(resp.text,'html.parser')
    l=soup.findAll("div",{"data-tid":"listing-info"})
    
    for listing in l:
        for link in listing:
            #Look for the link on a listing-info id; opens subsite
            if(link.get('href')):
                print("###")
                #Checking for adds, I had one apartment corrupt my data so I am checking for dupicates
                if ('https://www.apartmentguide.com'+link.get('href')) in subSiteList:
                    print("DUPLICATE!! NOT ADDED, PROBABLY ADD")
                else:
                    subSiteList.append('https://www.apartmentguide.com'+link.get('href'))
                    print('Following site added: {}'.format(link.get('href')))
        print('###')

In [ ]:
url = 'https://www.apartmentguide.com/apartments/Ohio/Dayton/'
listUrl = []
addUrls(url, listUrl)

In [ ]:
subSiteUrls = []
for url in listUrl:
    addSubSites(url, subSiteUrls)

In [ ]:
resp=requests.get('https://www.apartmentguide.com/apartments/Ohio/West-Carrollton/Oakwood-Apartments/7190/')         
if resp.status_code==200: 
    print("Successfully opened the web page") 
    #listUrl.append(url)
else:
    print("Did not work")
soup=BeautifulSoup(resp.text,'html.parser')

#amenities at the end of every break
amen=soup.find("div",{"data-tid":"features"})


for j in amen.findAll("div",{"class":"_1adc7"}):
    for word in j.findAll("div",{"class":"_2gvi2"}):
        print(word.text+'#')
    print('&&&#')

#pets at the end of every break
pets=soup.find("div",{"data-tid":"pets-section"})

for word in pets.findAll("div",{"class":"_2gvi2"}):
    print(word.text+'#')

In [ ]:
def gatherListingInfo(url, f):
    global globCount
    globCount = globCount + 1
    #open with GET method 
    resp=requests.get(url)         
    if resp.status_code==200: 
        print("Successfully opened the web page") 
        #listUrl.append(url)
    else:
        print("Did not work")
    soup=BeautifulSoup(resp.text,'html.parser')
    
    l=soup.findAll("div",{"data-tag_item":"floor_plans"})
    #for every sub layer of div tags loop;
    #loop through the general info if no sub entries are present
    for i in l:
        for j in i.findAll('div',{'data-tid':'name'}):
            subList = i.findAll('div',{'data-tag_item':"unit_title"})
            #Sublists are listed on website
            if len(subList) > 0:
                for item in subList:                    
                    #find and print ratings for apartments
                    l=soup.find("div",{"data-tid":"rating-stars"})
                    if l is not None:
                        for k in l.findChildren('div'):
                            for rating in k.findChildren('div'):
                                print(rating.get('style')[6:9])
                                #6:9 because the text is always a span element looking like <span>90%</span>
                                f.write(rating.get('style')[6:9]+'#')
                    else:
                        print("No ratings")
                        f.write('NA#')
                    #print and write to file -> apartment name
                    print('\n'+soup.find('h1',{'data-tid':'property-title'}).text)
                    f.write(soup.find('h1',{'data-tid':'property-title'}).text+'#')
                    #print apartment type
                    print(j.text)
                    f.write(j.text+'#')
                    for g in item:
                        mylist = []
                        for yolo in g.findChildren('span', recursive=False):
                            mylist.append(yolo.text)

                        if len(mylist) == 0:
                            #This serves for regex reasons later in the data preprocessing
                            if g.get('data-tid') == "fp-availability":
                                print('&'+g.text)
                                f.write('&'+g.text+'#')
                            else:
                                print(g.text)
                                f.write(g.text+'#')
                        else:
                            for item in mylist:
                                print(item)
                                f.write(item+'#')
                    print("AMENITIES")                    
                    #amenities at the end of every break
                    amen=soup.find("div",{"data-tid":"features"})
                    if amen is not None:
                        for j in amen.findAll("div",{"class":"_1adc7"}):
                            for word in j.findAll("div",{"class":"_2gvi2"}):                            
                                print(word.text+'#')
                                f.write(word.text+'#')
                            print('&&&#')
                            f.write('&&&#')
                    else:
                        print("No Amenitites")
                        f.write('NA#')
                    #pets at the end of every break
                    pets=soup.find("div",{"data-tid":"pets-section"})
                    if pets is not None: 
                        for word in pets.findAll("div",{"class":"_2gvi2"}):
                            print(word.text+'#')
                            f.write(word.text+'#')
                    print("BREAK")
                    f.write('\n')
            else:                
                
                for j in i.findAll('div',{"data-tid":"floorplan-row"}):
                    l=soup.find("div",{"data-tid":"rating-stars"})
                    if l is not None:
                        for k in l.findChildren('div'):
                            for rating in k.findChildren('div'):
                                print(rating.get('style')[6:9])
                                f.write(rating.get('style')[6:9]+'#')
                    else:
                        print("No ratings")
                        f.write('NA#')
                    print('\n'+soup.find('h1',{'data-tid':'property-title'}).text)
                    f.write(soup.find('h1',{'data-tid':'property-title'}).text+'#')
                    for p in j:
                        for prop in p:
                            mylist = []
                            for x in prop.findChildren('span', recursive = False):
                                mylist.append(x.text)
                            #span elements are empty which means bed and bath should be truncated within div tag    
                            if len(mylist) == 0:
                                if prop.get('data-tid') == "fp-availability":
                                    print('&'+prop.text)
                                    f.write(prop.text+'#')
                                else:
                                    print(prop.text)
                                    f.write(prop.text+'#')

                            else:
                                for item in mylist:
                                    print(item)
                                    f.write(item+'#')                
                    print("AMENITIES")                    
                    #amenities at the end of every break
                    amen=soup.find("div",{"data-tid":"features"})

                    if amen is not None:
                        for j in amen.findAll("div",{"class":"_1adc7"}):
                            for word in j.findAll("div",{"class":"_2gvi2"}):                            
                                print(word.text+'#')
                                f.write(word.text+'#')
                            print('&&&#')
                            f.write('&&&#')
                    else:
                        print("No Amenitites")
                        f.write('NA#')

                    #pets at the end of every break
                    pets=soup.find("div",{"data-tid":"pets-section"})
                    if pets is not None:    
                        for word in pets.findAll("div",{"class":"_2gvi2"}):
                            print(word.text+'#')
                            f.write(word.text+'#')                                            
                    print("BREAK")
                    
                    f.write('\n')
        print('\n')

    print("\n###Done with Website number {}###\n".format(globCount))          

In [ ]:
globCount = 0
f = open("finaltest.txt", "a")
for i in subSiteUrls:
    gatherListingInfo(i,f)
f.close()

In [ ]:
import csv

with open('finaltest.txt', 'r') as infile, open('TEST.csv', 'w') as outfile:
    stripped = (line.strip() for line in infile)
    lines = (line.split("#") for line in stripped if line)
    writer = csv.writer(outfile)
    writer.writerows(lines)